In [1]:
cd /content/drive/MyDrive/Data Mining

/content/drive/MyDrive/Data Mining


In [2]:
!pip install pytrends

In [3]:
import os
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from pytrends import dailydata
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
bitcoin_1_day_data = pd.read_csv('BTC-USDT-1h.csv')

In [ ]:
bitcoin_1_day_data.drop(columns = ['close_time'],inplace=True)

In [ ]:
bitcoin_1_day_data.head()

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181009,202366.138393,171,35.160503,150952.477943,7887.635513
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234916,100304.823567,102,21.448071,92608.279728,8039.262402
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229691,31282.312670,36,4.802861,20795.317224,8041.760498
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443249,19241.058300,25,2.602292,11291.347015,8048.127886
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,4239.503586,28,0.814655,3552.746817,8018.036500


In [ ]:
bitcoin_1_day_data['open_time'] = pd.to_datetime(bitcoin_1_day_data['open_time'],unit='ms').astype('str')


In [ ]:
bitcoin_1_day_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
open,37568.0,1.687740e+04,1.692789e+04,2870.9,6.693243e+03,9.224785e+03,1.643999e+04,6.863512e+04
high,37568.0,1.698188e+04,1.703116e+04,2950.0,6.721198e+03,9.259025e+03,1.661062e+04,6.900000e+04
low,37568.0,1.676488e+04,1.681737e+04,2817.0,6.660663e+03,9.185080e+03,1.630000e+04,6.845119e+04
close,37568.0,1.687855e+04,1.692852e+04,2919.0,6.694083e+03,9.224840e+03,1.644150e+04,6.863369e+04
volume,37568.0,2.115986e+03,2.192099e+03,0.0,9.216842e+02,1.549974e+03,2.592585e+03,4.725576e+04
quote_asset_volume,37568.0,4.304475e+07,7.106650e+07,0.0,6.970935e+06,1.483241e+07,4.920212e+07,1.514465e+09
number_of_trades,37568.0,3.133194e+04,3.780960e+04,0.0,7.846000e+03,1.708750e+04,4.224350e+04,7.992060e+05
taker_buy_base_asset_volume,37568.0,1.054088e+03,1.074575e+03,0.0,4.720572e+02,7.798739e+02,1.290401e+03,2.062879e+04
taker_buy_quote_asset_volume,37568.0,2.127792e+07,3.496107e+07,0.0,3.583444e+06,7.468385e+06,2.394153e+07,6.849674e+08
ignore,37568.0,1.616306e+03,7.001470e+03,0.0,0.000000e+00,0.000000e+00,0.000000e+00,7.992062e+04


In [ ]:
'''
google_trends_list = ['bitcoin chart',
 'bitcoin dollar',
 'bitcoin dolar',
 'bitcoin news',
 'bitcoin mining',
 'btc',
 'bitcoin usd',
 'bitcoin',
 'bitcoin to usd',
 'bitcoin price usd',
 'bitcoin stock',
 'bitcoin wallet',
 'bitcoin value',
 'cryptocurrency',
 'coinbase',
 'miner',
 'price bitcoin',
 'sell bitcoin',
 'sell btc',
 'what is bitcoin',
 'buy bitcoin']
# download daily trends data
from pytrends import dailydata
for topic in google_trends_list:
  print(topic)
  dailydata.get_daily_data(topic, 2017, 8, 2021, 12, geo = '').to_csv("google trends data/"+topic+'.csv')
'''

'\ngoogle_trends_list = [\'bitcoin chart\',\n \'bitcoin dollar\',\n \'bitcoin dolar\',\n \'bitcoin news\',\n \'bitcoin mining\',\n \'btc\',\n \'bitcoin usd\',\n \'bitcoin\',\n \'bitcoin to usd\',\n \'bitcoin price usd\',\n \'bitcoin stock\',\n \'bitcoin wallet\',\n \'bitcoin value\',\n \'cryptocurrency\',\n \'coinbase\',\n \'miner\',\n \'price bitcoin\',\n \'sell bitcoin\',\n \'sell btc\',\n \'what is bitcoin\',\n \'buy bitcoin\']\n# download daily trends data\nfrom pytrends import dailydata\nfor topic in google_trends_list:\n  print(topic)\n  dailydata.get_daily_data(topic, 2017, 8, 2021, 12, geo = \'\').to_csv("google trends data/"+topic+\'.csv\')\n'

In [ ]:
'''google_trends_dataframe = pd.DataFrame()
google_trends_list = ['buy bitcoin','sell bitcoin']
for keyword in google_trends_list:
  if not len(google_trends_dataframe):
    google_trends_dataframe = pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]]
  if len(google_trends_dataframe):
    google_trends_dataframe = pd.merge(google_trends_dataframe,pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]])
merged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on='open_time',right_on='date',how='outer')
merged_dataframe.drop(columns=['date'],inplace=True)'''

'google_trends_dataframe = pd.DataFrame()\ngoogle_trends_list = [\'buy bitcoin\',\'sell bitcoin\']\nfor keyword in google_trends_list:\n  if not len(google_trends_dataframe):\n    google_trends_dataframe = pd.read_csv("google trends data/"+keyword+\'.csv\')[[\'date\',keyword]]\n  if len(google_trends_dataframe):\n    google_trends_dataframe = pd.merge(google_trends_dataframe,pd.read_csv("google trends data/"+keyword+\'.csv\')[[\'date\',keyword]])\nmerged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on=\'open_time\',right_on=\'date\',how=\'outer\')\nmerged_dataframe.drop(columns=[\'date\'],inplace=True)'

In [ ]:
'''
#to download the google trends hourly data
google_trends_list = ['buy bitcoin','sell bitcoin']
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

pytrends.get_historical_interest(google_trends_list, year_start=2017,month_start=8, 
                                 day_start=1,hour_start=0, year_end=2017,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2017.csv')                       
pytrends.get_historical_interest(google_trends_list, year_start=2018,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2018,month_end=12,day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2018.csv')
pytrends.get_historical_interest(google_trends_list, year_start=2019,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2019,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2019.csv')
pytrends.get_historical_interest(google_trends_list, year_start=2020,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2020,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2020.csv')

pytrends.get_historical_interest(google_trends_list, year_start=2021,month_start=1, 
                                 day_start=1,hour_start=0, year_end=2021,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2021.csv')                                                                                                                                    
'''                      

"\n#to download the google trends hourly data\ngoogle_trends_list = ['buy bitcoin','sell bitcoin']\nfrom pytrends.request import TrendReq\n\npytrends = TrendReq(hl='en-US', tz=360)\n\npytrends.get_historical_interest(google_trends_list, year_start=2017,month_start=8, \n                                 day_start=1,hour_start=0, year_end=2017,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2017.csv')                       \npytrends.get_historical_interest(google_trends_list, year_start=2018,month_start=1, \n                                 day_start=1,hour_start=0, year_end=2018,month_end=12,day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_data_2018.csv')\npytrends.get_historical_interest(google_trends_list, year_start=2019,month_start=1, \n                                 day_start=1,hour_start=0, year_end=2019,month_end=12, day_end=31, hour_end=23,cat=0, geo='', gprop='', sleep=0).to_csv('hourly_trends_d

In [ ]:
google_trends_dataframe = []
for file in os.listdir('/content/drive/MyDrive/Data Mining/google trends hourly data/'):
  if file[:19] == "hourly_trends_data_":
    if len(google_trends_dataframe)==0:
      google_trends_dataframe.append(pd.read_csv('/content/drive/MyDrive/Data Mining/google trends hourly data/'+file))
    else:
      google_trends_dataframe.append(pd.read_csv('/content/drive/MyDrive/Data Mining/google trends hourly data/'+file))
google_trends_dataframe = pd.concat(google_trends_dataframe)
google_trends_dataframe.dropna(inplace=True)
google_trends_dataframe.drop(columns=['isPartial'],inplace=True)
bitcoin_1_day_data['open_time'] = bitcoin_1_day_data['open_time'].apply(lambda x:x[:-4])
merged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on='open_time',right_on='date',how='inner')
merged_dataframe.drop(columns=['date'],inplace=True)

In [ ]:
merged_dataframe.dropna(inplace=True)

In [ ]:
merged_dataframe['close']=merged_dataframe['close']>merged_dataframe['open']

In [ ]:
merged_dataframe.head()

,open_time,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,buy bitcoin,sell bitcoin
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,True,47.181009,202366.138393,171,35.160503,150952.477943,7887.635513,62,7
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,True,23.234916,100304.823567,102,21.448071,92608.279728,8039.262402,57,10
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,False,7.229691,31282.312670,36,4.802861,20795.317224,8041.760498,49,8
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,True,4.443249,19241.058300,25,2.602292,11291.347015,8048.127886,45,6
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,True,0.972807,4239.503586,28,0.814655,3552.746817,8018.036500,44,7


In [ ]:
n = len(merged_dataframe)
training_set = merged_dataframe.iloc[0:int(n*0.7),1:].values
test_set = merged_dataframe.iloc[int(n*0.7):,1:].values
num_features = training_set.shape[1]

In [ ]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
print(sc.fit(training_set))
training_set_scaled = sc.transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i,:])
    y_train.append(training_set_scaled[i,3])
X_train, y_train = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)

MinMaxScaler()


In [ ]:
test_set_scaled = sc.transform(test_set)
X_test = []
y_test = []
for i in range(60, len(test_set)):
    X_test.append(test_set_scaled[i-60:i, :])
    y_test.append(test_set_scaled[i,3])
X_test,y_test = tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)

In [ ]:
# Compiling the RNN linear
def compile_and_fit(model, X_train, y_train):
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[callback])
  return history


def compile_and_fit_log(model, X_train, y_train):
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  '''
  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  '''
  model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
  
  history = model.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[callback])
  return history

In [ ]:
google_trends_model = Sequential()
google_trends_model.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], num_features)))
google_trends_model.add(Dropout(0.2))
google_trends_model.add(LSTM(units = 120, return_sequences = True))
google_trends_model.add(Dropout(0.2))
google_trends_model.add(LSTM(units = 60))
google_trends_model.add(Dropout(0.2))
'''
google_trends_model.add(Dense(units = 1))
google_trends_model.add(Activation('linear'))
history = compile_and_fit(google_trends_model,X_train, y_train)
google_trends_model.save('my_model_with_1d_google_trends_price_prediction')
'''
google_trends_model.add(Dense(units = 1))
history = compile_and_fit_log(google_trends_model,X_train, y_train)
google_trends_model.save('my_model_with_1h_google_trends_price_change')
model = Sequential()
model.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], num_features-2)))
model.add(Dropout(0.2))
model.add(LSTM(units = 120, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 60))
model.add(Dropout(0.2))
'''
model.add(Dense(units = 1))
model.add(Activation('linear'))
history = compile_and_fit(model,X_train[:,:,:-2], y_train)
model.save('my_model_with_1d_without_google_trends_price_prediction')
'''
model.add(Dense(units = 1))
history = compile_and_fit_log(model,X_train[:,:,:-2], y_train)
model.save('my_model_with_1h_without_google_trends_price_change')

Epoch 1/100
758/758 [==============================] - 81s 100ms/step - loss: 0.6932 - accuracy: 0.4874
Epoch 2/100
758/758 [==============================] - 78s 103ms/step - loss: 0.6918 - accuracy: 0.4873
Epoch 3/100
758/758 [==============================] - 81s 107ms/step - loss: 0.6883 - accuracy: 0.4896
Epoch 4/100
758/758 [==============================] - 86s 114ms/step - loss: 0.6875 - accuracy: 0.4901
Epoch 5/100
758/758 [==============================] - 91s 120ms/step - loss: 0.6870 - accuracy: 0.4896
Epoch 6/100
758/758 [==============================] - 88s 116ms/step - loss: 0.6867 - accuracy: 0.4921
Epoch 7/100
758/758 [==============================] - 89s 118ms/step - loss: 0.6873 - accuracy: 0.4903
Epoch 8/100
758/758 [==============================] - 93s 123ms/step - loss: 0.6865 - accuracy: 0.4914
Epoch 9/100
758/758 [==============================] - 90s 118ms/step - loss: 0.6867 - accuracy: 0.4936
Epoch 10/100
758/758 [==============================] - 88s 117m

INFO:tensorflow:Assets written to: my_model_with_1h_google_trends_price_change/assets


INFO:tensorflow:Assets written to: my_model_with_1h_google_trends_price_change/assets


Epoch 1/100
758/758 [==============================] - 91s 113ms/step - loss: 0.6931 - accuracy: 0.4875
Epoch 2/100
758/758 [==============================] - 85s 113ms/step - loss: 0.6918 - accuracy: 0.4874
Epoch 3/100
758/758 [==============================] - 86s 113ms/step - loss: 0.6881 - accuracy: 0.4895
Epoch 4/100
758/758 [==============================] - 82s 108ms/step - loss: 0.6876 - accuracy: 0.4911
Epoch 5/100
758/758 [==============================] - 81s 107ms/step - loss: 0.6871 - accuracy: 0.4901
Epoch 6/100
758/758 [==============================] - 81s 106ms/step - loss: 0.6869 - accuracy: 0.4927
Epoch 7/100
758/758 [==============================] - 80s 106ms/step - loss: 0.6868 - accuracy: 0.4918
Epoch 8/100
758/758 [==============================] - 78s 103ms/step - loss: 0.6866 - accuracy: 0.4905
Epoch 9/100
758/758 [==============================] - 78s 102ms/step - loss: 0.6867 - accuracy: 0.4915
Epoch 10/100
758/758 [==============================] - 81s 107m

INFO:tensorflow:Assets written to: my_model_with_1h_without_google_trends_price_change/assets


INFO:tensorflow:Assets written to: my_model_with_1h_without_google_trends_price_change/assets


In [ ]:
#logistic 1 hour data
test_loss, test_acc = model.evaluate(X_test[:,:,:-2],  y_test, verbose=2)
test_loss, test_acc = google_trends_model.evaluate(X_test,  y_test, verbose=2)

324/324 - 11s - loss: 0.6930 - accuracy: 0.4918 - 11s/epoch - 33ms/step
324/324 - 11s - loss: 0.7041 - accuracy: 0.4917 - 11s/epoch - 32ms/step


In [4]:
bitcoin_1_day_data = pd.read_csv('BTC-USDT-1d.csv')
bitcoin_1_day_data.drop(columns = ['close_time'],inplace=True)
bitcoin_1_day_data['open_time'] = pd.to_datetime(bitcoin_1_day_data['open_time'],unit='ms').astype('str')
google_trends_dataframe = pd.DataFrame()
google_trends_list = ['buy bitcoin','sell bitcoin']
for keyword in google_trends_list:
  if not len(google_trends_dataframe):
    google_trends_dataframe = pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]]
  if len(google_trends_dataframe):
    google_trends_dataframe = pd.merge(google_trends_dataframe,pd.read_csv("google trends data/"+keyword+'.csv')[['date',keyword]])
merged_dataframe = pd.merge(bitcoin_1_day_data,google_trends_dataframe,left_on='open_time',right_on='date',how='outer')
merged_dataframe.drop(columns=['date'],inplace=True)
merged_dataframe.dropna(inplace=True)
merged_dataframe['close']=merged_dataframe['close']>merged_dataframe['open']
n = len(merged_dataframe)
training_set = merged_dataframe.iloc[0:int(n*0.7),1:].values
test_set = merged_dataframe.iloc[int(n*0.7):,1:].values
num_features = training_set.shape[1]
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
print(sc.fit(training_set))
training_set_scaled = sc.transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i,:])
    y_train.append(training_set_scaled[i,3])
X_train, y_train = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)
test_set_scaled = sc.transform(test_set)
X_test = []
y_test = []
for i in range(60, len(test_set)):
    X_test.append(test_set_scaled[i-60:i, :])
    y_test.append(test_set_scaled[i,3])
X_test,y_test = tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)
model = tf.keras.models.load_model('my_model_with_1d_without_google_trends_price_change')
google_trends_model = tf.keras.models.load_model('my_model_with_1d_google_trends_price_change')
test_loss, test_acc = model.evaluate(X_test[:,:,:-2],  y_test, verbose=2)
test_loss, test_acc = google_trends_model.evaluate(X_test,  y_test, verbose=2)

MinMaxScaler()
13/13 - 2s - loss: 0.6974 - accuracy: 0.4647 - 2s/epoch - 123ms/step
13/13 - 1s - loss: 0.7143 - accuracy: 0.4647 - 1s/epoch - 100ms/step


In [ ]:
'''
#linear
predicted_stock_price = model.predict(X_test[:,:,:-2])/sc.scale_[3]
predicted_stock_price_with_google_trends = google_trends_model.predict(X_test)/sc.scale_[3]
# Visualising the results
plt.figure(figsize=(18,16))
plt.plot(merged_dataframe.iloc[int(n*0.7)+60:,4].values, color = 'red', label = 'Real Bitcoin Price')
plt.plot(predicted_stock_price, color = 'blue', label = ' Predicted Price')
plt.plot(predicted_stock_price_with_google_trends, color = 'green', label = 'Predicted Price with google trends')
plt.xticks(np.arange(0,len(test_set[60:,3]),50))
plt.title('Bitcoin Price Prediction')
plt.xlabel('Time')
plt.ylabel('Bitcoin Price')
plt.legend()
plt.show()
Output_Df = pd.DataFrame()
original_close_Df = merged_dataframe.iloc[int(n*0.7)+60:,4].reset_index(drop = True)
Output_Df['original_close_percentage'] = (original_close_Df-original_close_Df.shift(1))/original_close_Df*100
predicted_stock_price_ = pd.Series(predicted_stock_price.reshape(1,-1)[0])
Output_Df['model_close_percentage'] = (predicted_stock_price_-predicted_stock_price_.shift(1))/predicted_stock_price_*100
predicted_stock_price_with_google_trends_ = pd.Series(predicted_stock_price_with_google_trends.reshape(1,-1)[0])
Output_Df['model_with_google_trends_close_percentage'] = (predicted_stock_price_with_google_trends_-predicted_stock_price_with_google_trends_.shift(1))/predicted_stock_price_with_google_trends_*100
Output_Df.dropna(inplace=True)
Output_Df['ori_gain']=Output_Df['original_close_percentage']>0
Output_Df['no_google_trends_gain']=Output_Df['model_close_percentage']>0
Output_Df['google_trends_gain']=Output_Df['model_with_google_trends_close_percentage']>0
print((Output_Df['ori_gain'] == Output_Df['no_google_trends_gain']).mean())
print((Output_Df['ori_gain'] == Output_Df['google_trends_gain']).mean())
'''